In [1]:
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
import matplotlib
import os
import sys
from cmcrameri import cm

# Add the parent directory to sys.path to find the utils package
module_path = os.path.abspath(os.path.join('..')) 
if module_path not in sys.path:
    sys.path.append(module_path)

from utils.colormaps import b_viridis, b_winter  

In [2]:
os.makedirs('result', exist_ok=True)

In [3]:
# Load data and rescale to physically meaningful original units
max_label = np.array([100, 0.17, 1e-4, 3.0])
min_label = np.array([5, 0.02, 1e-6, 0.2])
result_nn = sio.loadmat("result/quantification_UL.mat")['result_nn']
result_BE = np.transpose(np.reshape(result_nn, (27, 4, 256, 256)), (0, 2, 3, 1))
result_BE = result_BE * (max_label - min_label) + min_label
print("Quantitative maps shape: " + str(result_BE.shape))


Quantitative maps shape: (27, 256, 256, 4)


In [4]:
# Masking the background
mask = sio.loadmat('result/mask_mtc.mat')['mask']
mask_ = np.repeat(mask[:, :, :, np.newaxis], 4, axis=3)
result_BE[mask_<1] = 0


In [5]:
# Choosing slice to display (out of 27)
sl_num = 6
result_BE = result_BE[sl_num]
mask = mask[sl_num]

# Plot Rm
Rm_BE = result_BE[:, :, 0]
Rm_BE = Rm_BE[34:221, 47:212]
mask_ = mask[34:221, 47:212]

# Plot Mm
Mm_BE = result_BE[:, :, 1]
Mm_BE = Mm_BE[34:221, 47:212]

# Plot T2m
T2m_BE = result_BE[:, :, 2] * 1e6
T2m_BE = T2m_BE[34:221, 47:212]

# Plot T1w
T1w_BE = result_BE[:, :, 3]
T1w_BE = T1w_BE[34:221, 47:212]


In [7]:

plt.figure(figsize=(20,10))

pad_cb = 0.01  # colorbar padding
length_cb = 0.021  # colorbar length as a fraction of the axis
plt.rcParams['font.family'] = 'Arial'

matplotlib.rc('xtick', labelsize=20)
matplotlib.rc('ytick', labelsize=20)
plt.rcParams.update({'font.size': 25})

plt.rcParams['mathtext.fontset'] = 'custom' 
plt.rcParams['mathtext.rm'] = 'Arial' 
plt.rcParams['mathtext.it'] = 'Arial' 
plt.rcParams['mathtext.bf'] = 'Arial:bold'

plt.subplot(141)
plt.imshow(Rm_BE, vmin=5, vmax=50, cmap='magma')
plt.colorbar(orientation='horizontal', ticks=np.arange(5, 50 + 15, 15), pad=pad_cb, fraction=length_cb)
plt.title('k$_{ssw}$ (s$^{-1}$)')
plt.axis("off")

plt.subplot(142)
plt.imshow(Mm_BE, vmin=0.02, vmax=0.17, cmap=b_viridis)
plt.colorbar(orientation='horizontal', ticks=np.arange(0.02, 0.17 + 0.05, 0.05), pad=pad_cb, fraction=length_cb)
plt.title('M$_{0}$$^{ss}$ (M)')
plt.axis("off")

plt.subplot(143)
plt.imshow(T2m_BE, vmin=40, vmax=100, cmap=cm.navia)
plt.colorbar(orientation='horizontal', ticks=np.arange(0, 100 + 20, 20), pad=pad_cb, fraction=length_cb)
plt.title('T$_{2}$$^{ss}$ ($\mu$s)')
plt.axis("off")

plt.subplot(144)
plt.imshow(T1w_BE, vmin=0.5, vmax=2.5, cmap=cm.lipari)
plt.colorbar(orientation='horizontal', ticks=np.arange(0.5, 2.5 + 0.5, 0.5), pad=pad_cb, fraction=length_cb)
plt.title('Water T$_{1}$ (s)')
plt.axis("off")

plt.tight_layout()
plt.subplots_adjust(wspace=0.1, hspace=0)  # Adjust spacing between subplots
plt.savefig('result/human_results_unsupervised.eps', format='eps')
plt.savefig('result/human_results_unsupervised.pdf', format='pdf')
plt.show()